# DecisionTree Assignment - 20기 OOO

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

from functools import reduce # reduce (누적) 사용

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [3]:
def get_gini(df, label):
    
    # df 데이터의 전체 개수
    data_len = df[label].count()
    
    # 각 클래스별 count를 담은 Generator를 생성
    count_arr = (value for key, value in df[label].value_counts().items())
    
    # reduce lambda를 이용하여 지니 지수 계산
    gini = reduce(lambda x, y: x - (y/data_len)**2, count_arr, 1)
    
    return gini

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    
    # 결과에 대한 list 생성
    result = []
    
    # 조합(combinations) 이용해서 원소가 1, unique한 원소 개수보다 1개 작은 숫자까지 조합 생성
    for i in range(1, len(uniques)) :
        # 2^(n-1) - 1개 구해짐
        for att in combinations(uniques, i) :
            result.append(list(att))
    
    return result

In [6]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    
    # 결과 담을 dictionary 생성
    result = {}
    
    # 이진분류 생성
    binary_split = get_binary_split(df, attribute)
    
    # 데이터프레임 개수
    n = len(df)
    
    # 이진분류 반복문
    for col in binary_split :
        # dictionary에 담을 이름 선언(2개 이상일 경우 _으로 구분)
        col_name = "_".join(col)
        
        # 반전 연산(~)을 수행하기 위해서 numpy 배열 생성
        # attribute(속성)이 col에 속하면 할당
        mask = np.array(list(map(lambda x : x in col, df[attribute])))

        # 원하는 dataframe만 가져오기
        target = df.loc[mask, :]
        rest = df.loc[~mask, :]
        
        # 지니지수 계산 (t -> target // r -> rest)
        gini_t = get_gini(target, label)
        gini_r = get_gini(rest, label)
        
        # 최종적인 지니지수 계산
        gini = len(target) / n * gini_t + len(rest) / n * gini_r
        
        # 처음에 생성한 dictionary에 추가
        result[col_name] = gini
    return result

In [8]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [9]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4428571428571429)

In [10]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [11]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:

    # 해당 변수에 대해서 가장 낮은 gini 계수와 변수 출력
    min_gini = min(get_attribute_gini_index(pd_data, feature, label).items())[1]
    print(f"Minimum Gini index of {feature} : {min_gini :.4f}")
    

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [12]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [13]:
from math import log2

def getEntropy(df, feature) :
    
    
    """
    수식을 참고하여,
    데이터프레임 df에서 특정 feature에 대해 
    엔트로피를 구하는 함수를 작성해주세요.
    """
    
    # 데이터 전체 길이 할당
    data_len = df[feature].count()
    
    # reduce 함수를 이용하여 초기값 0에 대해서 각 feature 별 엔트로피 식에 대입한 값 누적으로 더하기
    return reduce(lambda x, y : x + (-(y[1]/data_len) * np.log2(y[1]/data_len)), df[feature].value_counts().items(), 0)

In [14]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [15]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
    
    # 결과값에 대한 초기 dictionary 생성    
    result = {}

    # 목표변수 feature에 대한 엔트로피 구하기
    info_D = getEntropy(df, feature)
    # 목표변수를 제외한 나머지 변수들 리스트 형태로 할당
    columns = list(df.loc[: , df.columns != feature])
    
    for col in columns : 
        # 가중치 초기화
        w = 0
        
        # 각 특성에 대한 고유값 반복
        for i in list(df[col].unique()) : 
            # 전체 데이터 개수 구하기
            data_len = len(df[feature])
            # 특성 col의 값이 i인 데이터 개수 구하기 (-> 가중치)
            data_len_i = sum(df[col] == i)
            
            # 가중치와 해당 특성 값에 대한 엔트로피의 곱을 누적하여 가중치 계산
            w += (data_len_i / data_len) * getEntropy(df[df[col] == i], feature)
        
        # col에 대한 정보 획득량 계산하여 저장 -> 정보의 양
        result[col] = info_D - w
    
    return(result)

In [16]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}

#### `정보 획득량이 가장 높은 변수는 age이다.`

# Assignment

## 우리의 목표 : 나이브베이즈 문제 해결하기
1.1) 입력문서가 {fast, furious, fun} 만을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 문서로 분류되는가?

##### 노트북 파일을 따라오면서, 빈칸과 질문에 대한 물음을 모두 채우시면 과제 완료 입니다 

### 주어진 dataset 을 확인해 보기위해 엑셀 파일을 읽습니다.

In [184]:
import numpy as np
import pandas as pd

In [185]:
documents = pd.read_excel('3wk_NaiveBayes_test_file.xlsx') 

In [186]:
documents 

,label,message
0,comedy,"Fun couple, love love"
1,action,"fast Furious, shoot!!"
2,comedy,"couple^^, fly, fast, fun, fun"
3,action,"furious, shoot shoot fun"
4,action,"fly~~ fast shoot, Love"


####  Q1. 발견한 **문제점**(전처리가 필요한 부분)을 2가지 이상 적어주세요.
- 1. `특수문자들을 제거해야 합니다.(ex ^^, ~~, !!)`
- 2. `구분이 잘 안 되어 있어서 띄어쓰기에 따라서 구분 지어 줘야 합니다.`

### step 1. 단어를 모두 소문자로 바꾸어 줍니다. 
#### Q2. 이 과정이 왜 필요할까요?
- A. `영어에서 대소문자를 구별하지 않으면, 다른 데이터를 인식할 수도 있기 때문에, 이에 대한 통합시키는 과정이 필요합니다.`

#### Q3. 코드속 빈칸을 채워주세요

In [187]:
lower_documents = []
lower_documents = [d.lower() for d in documents['message']]
print(lower_documents)

['fun couple, love love', 'fast furious, shoot!!', 'couple^^, fly, fast, fun, fun', 'furious, shoot shoot fun', 'fly~~ fast shoot, love']


### step2. 특수문자를 없애줍니다.
#### string.punctuation 에는 특수문자들이 저장되어있는것을 확인할수 있습니다.

In [188]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Q4. string.punctuation 을 이용해서 특수문자를 제거해주세요.

In [189]:
import re

del_punctuation_documents = []

for i in lower_documents:
    
    new_str = re.sub(f"[{string.punctuation}]", '', i)
    del_punctuation_documents.append(new_str)
    
del_punctuation_documents

['fun couple love love',
 'fast furious shoot',
 'couple fly fast fun fun',
 'furious shoot shoot fun',
 'fly fast shoot love']

### step 3. 단어를 하나씩 띄어쓰기 단위로 쪼개어 줍니다.
#### Q5. 빈칸을 채워주세요.

In [190]:
preprocessed_documents = [[w for w in d.split()] for d in del_punctuation_documents]
preprocessed_documents

[['fun', 'couple', 'love', 'love'],
 ['fast', 'furious', 'shoot'],
 ['couple', 'fly', 'fast', 'fun', 'fun'],
 ['furious', 'shoot', 'shoot', 'fun'],
 ['fly', 'fast', 'shoot', 'love']]

### step 4. 각각의 단어가 문서에서 몇번 나왔는지 세봅시다.

In [191]:
frequency_list = []
from collections import Counter

frequency_list = [Counter(d) for d in preprocessed_documents]
frequency_list

[Counter({'love': 2, 'fun': 1, 'couple': 1}),
 Counter({'fast': 1, 'furious': 1, 'shoot': 1}),
 Counter({'fun': 2, 'couple': 1, 'fly': 1, 'fast': 1}),
 Counter({'shoot': 2, 'furious': 1, 'fun': 1}),
 Counter({'fly': 1, 'fast': 1, 'shoot': 1, 'love': 1})]

### step 5. 이제 우리가 하고싶은것은, 문자를 숫자로 변환하는 작업입니다!(그림 참고)
- 이를 위해 CountVectorizer를 사용합니다.
- CountVectorizer는 문서 집합에서 단어 토큰을 생성하고, 각 단어 수를 세어 BOW (Bag of Words) 인코딩한 벡터를 만들어줍니다.

- 문서에 해당단어가 몇번 포함되었는지를 나타낼 때 사용하는 방법입니다.

- 참고 : https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/

![CountVectorized](https://user-images.githubusercontent.com/68625698/106378540-15d8ed80-63e9-11eb-8604-5c960c274867.PNG)

In [192]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
count_vector.fit(documents['message'])

CountVectorizer()

In [193]:
doc_array = count_vector.transform(documents['message']).toarray()
doc_array

array([[1, 0, 0, 1, 0, 2, 0],
       [0, 1, 0, 0, 1, 0, 1],
       [1, 1, 1, 2, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 2],
       [0, 1, 1, 0, 0, 1, 1]], dtype=int64)

In [194]:
count_vector.get_feature_names_out()

array(['couple', 'fast', 'fly', 'fun', 'furious', 'love', 'shoot'],
      dtype=object)

In [195]:
count_vector.vocabulary_

{'fun': 3,
 'couple': 0,
 'love': 5,
 'fast': 1,
 'furious': 4,
 'shoot': 6,
 'fly': 2}

In [196]:
frequency_matrix = pd.DataFrame(doc_array, columns = count_vector.get_feature_names_out())
frequency_matrix

,couple,fast,fly,fun,furious,love,shoot
0,1,0,0,1,0,2,0
1,0,1,0,0,1,0,1
2,1,1,1,2,0,0,0
3,0,0,0,1,1,0,2
4,0,1,1,0,0,1,1


In [197]:
frequency_matrix['count'] = frequency_matrix.sum(axis=1)

### step 6. 범주형 변수를 dummy변수로 변환해주는 작업(One-Hot Encoding!)을 해주어야합니다.
#### Q6. label 을 comedy =1, action =0 으로 변환해주세요

In [198]:
documents["label"]=pd.get_dummies(documents["label"]).comedy

In [199]:
doc = pd.concat([documents['label'],frequency_matrix],axis=1)

In [200]:
doc

,label,couple,fast,fly,fun,furious,love,shoot,count
0,True,1,0,0,1,0,2,0,4
1,False,0,1,0,0,1,0,1,3
2,True,1,1,1,2,0,0,0,5
3,False,0,0,0,1,1,0,2,4
4,False,0,1,1,0,0,1,1,4


###  step 7. 나이브 베이즈 계산을 해봅시다!

#### Q7. 입력문서가 {fast, furious, fun} 을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 문서로 분류가 될까요? ( 계산과정을 채워주세요) 

In [201]:
doc = np.array(doc)           # dataframe을 np-array로 변환해줍니다.
                         # return값은 np-matrix가 아니라 np-array입니다.
doc

array([[True, 1, 0, 0, 1, 0, 2, 0, 4],
       [False, 0, 1, 0, 0, 1, 0, 1, 3],
       [True, 1, 1, 1, 2, 0, 0, 0, 5],
       [False, 0, 0, 0, 1, 1, 0, 2, 4],
       [False, 0, 1, 1, 0, 0, 1, 1, 4]], dtype=object)

In [202]:
type(doc)

numpy.ndarray

####  P(Y=comedy), P(Y=action) 계산하기

In [203]:
# P(Y=comedy)
p_comedy = sum(doc[:,0]==1) / len(doc)

# P(Y=action)
p_action = 1 - p_comedy

In [204]:
print('p_comedy : ',p_comedy)
print('p_action : ', p_action)

p_comedy :  0.4
p_action :  0.6


#### P(fast=1|comedy=1), P(furious=1|comedy=1), P(fun=1|comedy=1) 계산하기
* 참고 : 문서에 두번 등장한 단어 주의

*  조건부 확률 분모 구하기
`doc`의 `count` 위치 : doc array의 8번 째 열 <br>
-> sum(doc[doc[:, 0] == 1][:, 8]) : 장르가 comedy(1)에 속하는 전체 단어 개수

*  조건부 확률 분자 구하기
1. `doc`의 `fast` 위치 : doc array의 2번 째 열 <br>
-> sum(doc[doc[:, 0] == 1][:, 2]) : 장르가 comedy(1)인 fast 수
2. `doc`의 `furious` 위치 : doc array의 5 째 열 <br>
-> sum(doc[doc[:, 0] == 1][:, 5]) : 장르가 comedy(1)인 fly 수 
3. `doc`의 `fun` 위치 : doc array의 4번 째 열 <br>
-> sum(doc[doc[:, 0] == 1][:, 4]) : 장르가 comedy(1)인 shoot 수

In [205]:
# P(fast=1|comedy=1)
p_comedy_fast = sum( doc[doc[:, 0] == 1][:, 2]) / sum(doc[doc[:, 0] == 1][:, 8])

# P(furious=1|comedy=1)
p_comedy_furious = sum(doc[doc[:, 0] == 1][:, 5]) / sum(doc[doc[:, 0] == 1][:, 8]) 

# P(fun=1|comedy=1)
p_comedy_fun = sum(doc[doc[:, 0] == 1][:, 4]) / sum(doc[doc[:, 0] == 1][:, 8])

In [206]:
print('p_comedy_fast : ' , p_comedy_fast)
print('p_comedy_furious : ' , p_comedy_furious)
print('p_comedy_fun : ' , p_comedy_fun)   

p_comedy_fast :  0.1111111111111111
p_comedy_furious :  0.0
p_comedy_fun :  0.3333333333333333


#### P(fast=1|action=1), P(furious=1|action=1), P(fun=1|action=1) 계산하기
* 참고 : 문서에 두번 등장한 단어 주의

*  조건부 확률 분모 구하기
`doc`의 `count` 위치 : doc array의 8번 째 열 <br>
-> sum(doc[doc[:, 0] == 0][:, 8]) : 장르가 action(0)에 속하는 전체 단어 개수

*  조건부 확률 분자 구하기
1. `doc`의 `fast` 위치 : doc array의 2번 째 열 <br>
-> sum(doc[doc[:, 0] == 0][:, 2]) : 장르가 action(0)인 fast 수
2. `doc`의 `furious` 위치 : doc array의 5번 째 열 <br>
-> sum(doc[doc[:, 0] == 0][:, 5]) : 장르가 action(0)인 furious 수 
3. `doc`의 `fun` 위치 : doc array의 4번 째 열 <br>
-> sum(doc[doc[:, 0] == 0][:, 4]) : 장르가 action(0)인 fun 수

In [207]:
# P(fast=1|action=1)
p_action_fast = sum(doc[doc[:, 0] == 0][:, 2]) / sum(doc[doc[:, 0] == 0][:, 8])

# P(furious=1|action=1)
p_action_furious = sum(doc[doc[:, 0] == 0][:, 5]) / sum(doc[doc[:, 0] == 0][:, 8])

# P(fun=1|action=1)
p_action_fun = sum(doc[doc[:, 0] == 0][:, 4]) / sum(doc[doc[:, 0] == 0][:, 8])

In [208]:
print('p_action_fast : ' , p_action_fast)
print('p_action_furious : ' , p_action_furious)
print('p_action_fun : ' , p_action_fun)  

p_action_fast :  0.18181818181818182
p_action_furious :  0.18181818181818182
p_action_fun :  0.09090909090909091


#### P(Y = comedy| X = fast, furious, fun) , P(Y = action=1| X = fast, furious, fun) 값 계산하기

In [209]:
#P(Y = comedy| X = fast, furious, fun)
proba_comedy = p_comedy_fast * p_comedy_furious * p_comedy_fun * (2/5)

#P(Y = action=1| X = fast, furious, fun)
proba_action = p_action_fast * p_action_furious * p_action_fun * (3/5)

In [210]:
print('proba_comedy', proba_comedy)
print('proba_action', proba_action)

proba_comedy 0.0
proba_action 0.0018031555221637867


### step 8. 다음 값을 비교해봅시다.
1. P(Y = comedy| X = fast, furious, fun)
2. P(Y = action=1| X = fast, furious, fun)


Q7. 어떤 문제점을 발견할수 있나요? 문제점을 해결하기 위한 방법으로는 어떤게 있을까요?
- A. likelihood가 0이 되는 문제점이 발생할 수 있다. (한 번도 일어나지 않은 경우가 있기 때문에) <br>
- 따라서 이를 해결하기 위해서는 `라플라스 스무딩`을 통해 최소 한 번은 발생한 것으로 간주하여, 최소한의 확률을 구한다.